In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from collections import defaultdict
from json import JSONDecodeError
from qwikidata.sparql import return_sparql_query_results # python library for working with sparql and linked data from WikiData
import time
from requests.exceptions import ChunkedEncodingError
import math
#from SPARQLWrapper import SPARQLWrapper, JSON # sparql query library
import ssl

#  Explore csv
Meglio tenere il corso_codice perchè è la foreign key di DegreeProgrammes e CourseOfStudies (educational activities): serve per unire i due database per facilitare le relazioni successive. 

In [3]:
# create first dataframe only using the specified columns 
dataDegree = pd.read_csv("DegreeProgrammes.csv", sep=",",
                    usecols=["annoaccademico", "corso_codice", "corso_descrizione", "url", "campus", "sededidattica", "ambiti", "tipologia", "durata", "internazionale", "internazionale_titolo", "accesso", "lingue"], encoding="utf-8")

dataDegree.to_pickle("00_first_Degree.pickle")

dataDegree.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,lingue,accesso
0,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,Italiano,libero accesso con verifica delle conoscenze
1,660,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Ravenna,Ravenna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,Italiano,libero accesso con verifica delle conoscenze
2,881,2018/2019,Architettura,http://www.unibo.it/it/didattica/corsi-di-stud...,Cesena,Cesena,Ingegneria e architettura,Laurea Magistrale a ciclo unico,5,False,NaN,Italiano,numero chiuso
3,884,2018/2019,Chimica industriale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Scienze,Laurea Magistrale,2,True,Titolo multiplo (solo alcuni curricula),Italiano,libero accesso con verifica dell'adeguatezza d...
4,897,2018/2019,Direzione aziendale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Economia e management,Laurea Magistrale,2,True,NaN,Italiano Inglese,numero chiuso


In [4]:
data_mask = dataDegree['ambiti'] == "Studi umanistici"
dataHumanitiesDegree = dataDegree[data_mask].reset_index(drop=True)
dataHumanitiesDegree.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,lingue,accesso
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze
1,957,2018/2019,Filosofia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze
2,962,2018/2019,Storia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze
3,964,2018/2019,Antropologia culturale ed etnologia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea Magistrale,2,False,NaN,Italiano,libero accesso con verifica dell'adeguatezza d...
4,966,2018/2019,"Cinema, televisione e produzione multimediale",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea Magistrale,2,False,NaN,Italiano,libero accesso con verifica dell'adeguatezza d...


In [5]:
dataHumanitiesDegree.to_pickle("00_Humanities_Degree.pickle")

Ora leggiamo il db dedicato alle educational activities, di cui integriamo materia_codice e descrizione, usando corso_codice come key in comune. 

In [6]:
dataCourse = pd.read_csv("CoursesOfStudies.csv", sep=",",
                    usecols=["corso_codice", "materia_codice", "materia_descrizione"], encoding="utf-8")

dataCourse.to_pickle("00_first_Course.pickle")

dataCourse.head()

,corso_codice,materia_codice,materia_descrizione
0,8894,81644,LABORATORY ON RENEWABLE RAW MATERIALS AND CIRC...
1,9222,85238,BAYESIAN INFERENCE
2,8420,87462,HISTORY OF ECONOMIC THOUGHT
3,9222,85199,SURVEY SAMPLING
4,8412,87608,TECNOLOGIA E LEGISLAZIONE FARMACEUTICHE


In [7]:
dataHumanitiesDegree["corso_codice"] = dataHumanitiesDegree["corso_codice"].astype(str)
dataCourse["corso_codice"] = dataCourse["corso_codice"].astype(str)
dataCourse["materia_codice"] = dataCourse["materia_codice"].astype(str)
data = pd.merge(dataHumanitiesDegree, dataCourse, on="corso_codice", how="left")

In [8]:
data["Italian University"] = "University of Bologna"

In [9]:
data.to_pickle("00_data.pickle")
pd.set_option("display.max_rows", None)
data.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,lingue,accesso,materia_codice,materia_descrizione,Italian University
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna


Fino a qui tutto bene

In [10]:
data = pd.read_pickle("00_data.pickle")

In [11]:
SSDandCFU = pd.read_csv("DettagliCurriculaSSDeCFU.csv", sep=",",
                        usecols=["materia_codice", "insegnamento_ssd", "insegnamento_crediti"], encoding="utf-8")
#SSDandCFU["curriculum_codice"] = SSDandCFU["curriculum_codice"].astype(str)
SSDandCFU.head()

,materia_codice,insegnamento_ssd,insegnamento_crediti
0,222.0,IUS/08,12.0
1,74764.0,IUS/01,12.0
2,536.0,IUS/18,8.0
3,24331.0,IUS/19,8.0
4,230.0,IUS/13,9.0


# Togliamo tutti i duplicati di valore dei codici materia 

In [12]:
DDD = SSDandCFU.drop_duplicates(subset=['materia_codice'], keep='first')

In [13]:
DDD["materia_codice"] = DDD["materia_codice"].astype(str)
DDD["insegnamento_crediti"] = DDD["insegnamento_crediti"].astype(str)
DDD.head()

C:\Users\bordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\bordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,materia_codice,insegnamento_ssd,insegnamento_crediti
0,222.0,IUS/08,12.0
1,74764.0,IUS/01,12.0
2,536.0,IUS/18,8.0
3,24331.0,IUS/19,8.0
4,230.0,IUS/13,9.0


Facciamo una funzione che tolga il .0 dai nostri valori 

In [16]:
def reformat_materia_codice(name):
    name = name.replace(".0", "")
    return name

In [17]:
DDD["materia_codice"] = DDD["materia_codice"].apply(reformat_materia_codice)
DDD["insegnamento_crediti"] = DDD["insegnamento_crediti"].apply(reformat_materia_codice)
DDD.head(200)

C:\Users\bordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\bordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,materia_codice,insegnamento_ssd,insegnamento_crediti
0,222,IUS/08,12
1,74764,IUS/01,12
2,536,IUS/18,8
3,24331,IUS/19,8
4,230,IUS/13,9
5,40344,IUS/13,9
6,385,IUS/20,9
7,32616,IUS/20,9
8,26356,NaN,5
9,26338,NaN,5


In [18]:
def reformat_names(name):
    new = re.sub(r"\s+", "", name)
    return new

In [19]:
DDD["materia_codice"] = DDD["materia_codice"].apply(reformat_names)
data["materia_codice"] = data["materia_codice"].apply(reformat_names)

C:\Users\bordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dataPlusCurricula = pd.merge(data, DDD, on="materia_codice", how="left")

Alcuni valori di ssd e cfu non sono presenti in quanto NON disponibili all'interno del database

In [21]:
pd.set_option("display.max_rows", None)
dataPlusCurricula.head(200)

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,lingue,accesso,materia_codice,materia_descrizione,Italian University,insegnamento_ssd,insegnamento_crediti
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna,L-ART/07,6
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna,L-ART/06,12
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna,M-DEA/01,12
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna,L-ART/06,6
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna,L-ART/02,6
5,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,3510,DRAMMATURGIA,University of Bologna,L-ART/05,12
6,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,3522,DRAMMATURGIA MUSICALE,University of Bologna,L-ART/07,12
7,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,78048,ELEMENTI DI ARMONIA E CONTRAPPUNTO,University of Bologna,L-ART/07,12
8,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,345,ESTETICA,University of Bologna,M-FIL/04,12
9,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,3527,ETNOMUSICOLOGIA,University of Bologna,L-ART/08,12


# COSE CHE MANCANO DA INTEGRARE
Ora, procediamo con integrare la DegreeClass e la DegreeClassDescription come ulteriori colonne del database, associate ad ogni codice corso. 

Riflettere sulla INTERNATIONALQualification e sugli individui di titolo congiunto O multiplo (aggiungere un ID?) 

Integrare classi di laurea e description di classi di laurea 

Dipartimenti = tenere nell'ontologia (schema formale) ma non integrare nel database

Vedere se ci sono ontologie da poter riutilizzare

1. Trasformare il db in turtle
2. Popolare il database 

In [22]:
#rename columns
dataPlusCurricula.rename(columns = {'corso_codice':'id', 'annoaccademico':'academicYear', 'url':'DegreeProgramme', 'corso_descrizione':'DegreeProgrammeLabel', 'sededidattica':'PlaceOfTeaching', 'ambiti':'AcademicField', 'tipologia':'type', 'durata':'hasYearsOfDuration', 'internazionale':'International', 'internazionale_titolo':'InternationalQualification', 'lingue':'TeachingLanguage', 'accesso':'AccessType', 'materia_codice':'TeachingCode', 'materia_descrizione':'EducationalActivity', 'insegnamento_ssd' : 'Ssd', 'insegnamento_crediti' : 'Cfu'}, inplace = True)

In [23]:
dataPlusCurricula.to_pickle("01_dataRename.pickle")

Parti da qui 

In [37]:
data = pd.read_pickle("01_dataRename.pickle")

Integrazione semi-automatica delle classi di laurea e delle loro descrizioni

In [38]:
def degreeClass(value):
    degreeClassValue = ''
    if value == '956':
        degreeClassValue = 'L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA MUSICA, DELLO SPETTACOLO E DELLA MODA'
    if value == '957':
        degreeClassValue = 'L-5, FILOSOFIA'
    if value == '962':
        degreeClassValue = 'L-42, STORIA'
    if value == '964':
        degreeClassValue = 'LM-1, ANTROPOLOGIA CULTURALE ED ETNOLOGIA'
    if value == '966':
        degreeClassValue = 'LM-65, SCIENZE DELLO SPETTACOLO E PRODUZIONE MULTIMEDIALE'
    if value == '970':
        degreeClassValue = 'LM-15, FILOLOGIA, LETTERATURE E STORIA DELL ANTICHITÀ'
    if value == '971':
        degreeClassValue = 'LM-80, SCIENZE GEOGRAFICHE'
    if value == '977':
        degreeClassValue = 'LM-89, STORIA DELL ARTE'
    if value == '8493':
        degreeClassValue = 'L-42, STORIA'
    if value == '8616':
        degreeClassValue = 'LMR/02, CONSERVAZIONE E RESTAURO DEI BENI CULTURALI'
    if value == '8772':
        degreeClassValue = 'L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA MUSICA, DELLO SPETTACOLO E DELLA MODA'
    if value == '8773':
        degreeClassValue = 'LM-78, SCIENZE FILOSOFICHE'
    if value == '8837':
        degreeClassValue = 'LM-45, MUSICOLOGIA E BENI MUSICALI / LM-65, SCIENZE DELLO SPETTACOLO E PRODUZIONE MULTIMEDIALE'
    if value == '8838':
        degreeClassValue = 'LM-5, ARCHIVISTICA E BIBLIOTECONOMIA'
    if value == '8845':
        degreeClassValue = 'LM-84, SCIENZE STORICHE'
    if value == '8849':
        degreeClassValue = 'L-1, BENI CULTURALI'
    if value == '8850':
        degreeClassValue = 'L-10, LETTERE'
    if value == '8855':
        degreeClassValue = 'LM-2, ARCHEOLOGIA'
    if value == '8885':
        degreeClassValue = 'L-20, SCIENZE DELLA COMUNICAZIONE'
    if value == '8886':
        degreeClassValue = 'LM-92, TEORIE DELLA COMUNICAZIONE'
    if value == '9022':
        degreeClassValue = 'LM-65, SCIENZE DELLO SPETTACOLO E PRODUZIONE MULTIMEDIALE'
    if value == '9216':
        degreeClassValue = 'L-5, FILOSOFIA'
    if value == '9218':
        degreeClassValue = 'LM-2, ARCHEOLOGIA / LM-89, STORIA DELL ARTE'
    if value == '9220':
        degreeClassValue = 'LM-14, FILOLOGIA MODERNA'
    if value == '9224':
        degreeClassValue = 'LM-43, METODOLOGIE INFORMATICHE PER LE DISCIPLINE UMANISTICHE'
    return degreeClassValue

In [39]:
data["DegreeClass"] = data["id"].apply(degreeClass)
data.head()

,id,academicYear,DegreeProgrammeLabel,DegreeProgramme,campus,PlaceOfTeaching,AcademicField,type,hasYearsOfDuration,International,InternationalQualification,TeachingLanguage,AccessType,TeachingCode,EducationalActivity,Italian University,Ssd,Cfu,DegreeClass
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna,L-ART/07,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M..."
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna,L-ART/06,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M..."
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna,M-DEA/01,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M..."
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna,L-ART/06,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M..."
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna,L-ART/02,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M..."


Ora trasformiamo i Cfu e gli anni di durata in integer per permettere il count nelle queries sparql 

In [40]:
data['Cfu'] = pd.to_numeric(data['Cfu'], errors='coerce')
data = data.dropna(subset=['Cfu'])

In [43]:
data["Cfu"] = data["Cfu"].astype(int)
data["hasYearsOfDuration"] = data["hasYearsOfDuration"].astype(int)

In [44]:
data.to_pickle("00_dataClassDegree.pickle")

In [57]:
data = pd.read_pickle("00_dataClassDegree.pickle")

Ora aggiungiamo i dipartimenti

In [58]:
def department(value):
    department = ''
    if value == '956':
        department = 'delle Arti - DARvipem'
    if value == '957':
        department = 'Filosofia e Comunicazione - FILCOM'
    if value == '962':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '964':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '966':
        department = 'delle Arti - DARvipem'
    if value == '970':
        department = 'Filologia Classica e Italianistica - FICLIT'
    if value == '971':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '977':
        department = 'delle Arti - DARvipem'
    if value == '8493':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '8616':
        department = 'Beni Culturali - DBC'
    if value == '8772':
        department = 'delle Arti - DARvipem'
    if value == '8773':
        department = 'Filosofia e Comunicazione - FILCOM'
    if value == '8837':
        department = 'delle Arti - DARvipem'
    if value == '8838':
        department = 'Beni Culturali - DBC'
    if value == '8845':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '8849':
        department = 'Beni Culturali - DBC'
    if value == '8850':
        department = 'Filologia Classica e Italianistica - FICLIT'
    if value == '8855':
        department = 'Storia Culture Civiltà - DiSCi'
    if value == '8885':
        department = 'delle Arti - DARvipem'
    if value == '8886':
        department = 'Filosofia e Comunicazione - FILCOM'
    if value == '9022':
        department = 'Arts - DARvipem'
    if value == '9216':
        department = 'Filosofia e Comunicazione - FILCOM'
    if value == '9218':
        department = 'Beni Culturali - DBC'
    if value == '9220':
        department = 'Filologia Classica e Italianistica - FICLIT'
    if value == '9224':
        department = 'Classical Philology and Italian Studies - FICLIT'
    return department

In [59]:
data["Department"] = data["id"].apply(department)

In [60]:
data.to_pickle("00_dataDepartments.pickle")
data.head()

,id,academicYear,DegreeProgrammeLabel,DegreeProgramme,campus,PlaceOfTeaching,AcademicField,type,hasYearsOfDuration,International,InternationalQualification,TeachingLanguage,AccessType,TeachingCode,EducationalActivity,Italian University,Ssd,Cfu,DegreeClass,Department
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna,L-ART/07,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna,L-ART/06,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna,M-DEA/01,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna,L-ART/06,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,Italiano,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna,L-ART/02,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem


Vorrei ora provare a distinguere in modo semiautomatico il tipo di attività richieste da ogni corso di studio 

In [24]:
data = pd.read_pickle("00_dataDepartments.pickle")

In [25]:
def typeOfActivity(value):
    typeOfActivity = ''
    value = value.lower()
    if ('tirocinio' in value) or ('internship' in value):
        typeOfActivity = 'Internship'
    elif ('laboratorio' in value) or ('laboratory' in value):
        typeOfActivity = 'Laboratory'
    else:
        if ('seminario' in value) or ('seminar' in value):
            typeOfActivity = 'Seminar'
        else:
            typeOfActivity = 'Exam'
    return typeOfActivity

In [26]:
data["typeOfActivity"] = data["EducationalActivity"].apply(typeOfActivity)

In [27]:
pd.set_option("display.max_rows", None)
data.head()

,id,academicYear,DegreeProgrammeLabel,DegreeProgramme,campus,PlaceOfTeaching,AcademicField,type,hasYearsOfDuration,International,...,TeachingLanguage,AccessType,TeachingCode,EducationalActivity,Italian University,Ssd,Cfu,DegreeClass,Department,typeOfActivity
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,...,Italiano,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna,L-ART/07,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem,Exam
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,...,Italiano,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna,L-ART/06,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem,Exam
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,...,Italiano,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna,M-DEA/01,12,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem,Exam
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,...,Italiano,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna,L-ART/06,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem,Exam
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,...,Italiano,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna,L-ART/02,6,"L-3, DISCIPLINE DELLE ARTI FIGURATIVE, DELLA M...",delle Arti - DARvipem,Exam


In [14]:
#print(data.loc[data['typeOfActivity'] == 'Laboratory', 'EducationalActivity'])

In [15]:
#print(data.loc[data['typeOfActivity'] == 'Seminar', 'EducationalActivity'])

In [28]:
data.to_pickle("00_dataTypeOfActivity.pickle")

In [29]:
data.to_csv("FinalData.csv")